In [49]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
from math import factorial as ft 

In [50]:
car_data = pd.read_csv('/Users/leahdeyoung/Desktop/GitHub/car-data-project-practicum/vehicles_us.csv', encoding = "utf-8")

display(car_data.head())
car_data.info()


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [51]:
print(car_data['model_year'].isna().sum())
car_data['model_year'] = car_data['model_year'].fillna(0)
car_data['model_year'] = car_data['model_year'].astype('int')
print(car_data['model_year'].isna().sum())

3619
0


In [52]:
print(car_data['paint_color'].isna().sum())
car_data['paint_color'] = car_data['paint_color'].fillna('Unknown')
print(car_data['paint_color'].isna().sum())

9267
0


In [53]:
print(car_data['cylinders'].isna().sum())
car_data['cylinders'] = car_data['cylinders'].fillna(0)
car_data['cylinders'] = car_data['cylinders'].astype('int')
print(car_data['cylinders'].isna().sum())

5260
0


In [54]:
print(car_data['odometer'].isna().sum())
car_data['odometer'] = car_data['odometer'].fillna(111111111)
car_data['odometer'] = car_data['odometer'].astype('int')
print(car_data['odometer'].isna().sum())

7892
0


In [55]:
print(car_data['is_4wd'].isna().sum())
car_data['is_4wd'] = car_data['is_4wd'].fillna(0)
car_data['is_4wd'] = car_data['is_4wd'].astype('int')
print(car_data['is_4wd'].isna().sum())

25953
0


In [57]:
display(car_data.sample(10))
car_data.info()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
13904,7900,2009,hyundai sonata,excellent,4,gas,97012,automatic,sedan,grey,0,2018-11-12,3
42158,20995,2013,ford f150,like new,6,gas,111111111,automatic,truck,grey,1,2019-01-01,29
25661,3500,2015,ford explorer,excellent,6,gas,90000,automatic,SUV,red,0,2018-07-17,5
5766,4695,2007,nissan sentra,excellent,4,gas,135000,automatic,sedan,red,0,2018-12-07,7
39746,10800,2012,jeep grand cherokee,good,6,gas,122000,automatic,SUV,Unknown,0,2018-12-05,5
17680,11490,2017,chevrolet cruze,excellent,0,gas,62621,automatic,sedan,black,0,2018-10-25,52
36674,3900,2012,ford fusion se,like new,4,gas,178000,automatic,sedan,white,0,2018-05-12,24
13979,69,2018,toyota rav4,excellent,4,gas,111111111,automatic,SUV,grey,0,2018-06-12,17
36354,12990,2011,ford expedition,good,0,gas,124775,automatic,SUV,Unknown,1,2019-01-11,100
4899,16803,2015,hyundai santa fe,excellent,0,gas,76420,automatic,SUV,silver,0,2019-03-02,101


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   price         51525 non-null  int64 
 1   model_year    51525 non-null  int64 
 2   model         51525 non-null  object
 3   condition     51525 non-null  object
 4   cylinders     51525 non-null  int64 
 5   fuel          51525 non-null  object
 6   odometer      51525 non-null  int64 
 7   transmission  51525 non-null  object
 8   type          51525 non-null  object
 9   paint_color   51525 non-null  object
 10  is_4wd        51525 non-null  int64 
 11  date_posted   51525 non-null  object
 12  days_listed   51525 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 5.1+ MB


In [61]:
print(car_data['fuel'].unique())
print(car_data['transmission'].unique())
print(car_data['type'].unique())
print(car_data['price'].unique())

['gas' 'diesel' 'other' 'hybrid' 'electric']
['automatic' 'manual' 'other']
['SUV' 'pickup' 'sedan' 'truck' 'coupe' 'van' 'convertible' 'hatchback'
 'wagon' 'mini-van' 'other' 'offroad' 'bus']
[ 9400 25500  5500 ...  3636  9249  7455]
